# nnAudio

Audio processing by using pytorch 1D convolution network. By doing so, spectrograms can be generated from audio on-the-fly during neural network training. Kapre and torch-stft have a similar concept in which they also use 1D convolution from keras adn PyTorch to do the waveforms to spectrogram conversions.

Other GPU audio processing tools are torchaudio and tf.signal. But they are not using the neural network approach, and hence the Fourier basis can not be trained.

The name of nnAudio comes from torch.nn, since most of the codes are built from torch.nn.

In [ ]:
!pip install nnAudio

In [ ]:
from nnAudio import Spectrogram
from scipy.io import wavfile
import torch
import matplotlib.pyplot as plt

In [ ]:
if torch.cuda.is_available():
    device = "cuda:0"
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    device = "cpu"

In [ ]:
!wget "https://file-examples-com.github.io/uploads/2017/11/file_example_WAV_1MG.wav"
sr, song = wavfile.read('file_example_WAV_1MG.wav') # Loading your audio
x = song.mean(1) # Converting Stereo  to Mono
x = torch.tensor(x, device=device).float() # casting the array into a PyTorch Tensor

**Currently there are 3 models to generate various types of spectrograms**

# 1. STFT

In [ ]:
# linear spectrograms
spec_layer = Spectrogram.STFT(n_fft=2048, freq_bins=None, hop_length=512, 
                              window='hann', freq_scale='log', center=True, pad_mode='reflect', 
                              fmin=50,fmax=11025, sr=sr, device=device) # Initializing the model with device='cuda:0'
spec_lin = spec_layer(x)

In [ ]:
# log spectrograms
spec_layer = Spectrogram.STFT(n_fft=2048, freq_bins=None, hop_length=512, 
                              window='hann', freq_scale='linear', center=True, pad_mode='reflect', 
                              fmin=50,fmax=11025, sr=sr, device=device) # Initializing the model with device='cuda:0'
spec_log = spec_layer(x)

# 2. Mel Spectrogram

In [ ]:
# Mel spectrogram
spec_layer = Spectrogram.MelSpectrogram(sr=22050, n_fft=2048, n_mels=128, 
                                        hop_length=512, window='hann', center=True, 
                                        pad_mode='reflect', htk=False, fmin=0.0, fmax=None, 
                                        norm=1, trainable_mel=False, trainable_STFT=False, device=device)  # Initializing the model with device='cuda:0'
spec_mel = spec_layer(x)

# 3. CQT

In [ ]:
# CQT
spec_layer = Spectrogram.CQT2010v2(sr=22050, hop_length=512, fmin=55, fmax=None, n_bins=88,
                                   bins_per_octave=12, norm=True, basis_norm=1, 
                                   window='hann', pad_mode='reflect', earlydownsample=True, device=device)  # Initializing the model with device='cuda:0'

spec_cqt = spec_layer(x)

In [ ]:
fig, ax = plt.subplots(1,4, figsize=(16,3))
labels = ['spec_lin', 'spec_log', 'spec_mel', 'spec_cqt']
for i,spec in enumerate([spec_lin, spec_log, spec_mel, spec_cqt]):
  ax[i].imshow(spec.cpu().numpy()[0], origin='lower', cmap='jet', aspect='auto')
  ax[i].title.set_text(labels[i])

Source: https://github.com/KinWaiCheuk/nnAudio